# Latent Dirichlet Allocation

Following Mirko Draca and Carlo Schwarz selection of WVS questions and waves

In [ ]:
#pip install -r requirements.txt

In [1]:
import os
import pandas as pd

df_wvs = pd.read_csv('../master_thesis_R/wvs_ts_w1_w7.csv')

In [9]:
import pandas as pd

def recode_survey_responses(df, question_columns, neutral_values={3, 5}):
    """
    Recodes survey responses into two indicator variables (support and oppose), 
    imputes missing values with the sample mean of the non-missing data in the same wave,
    and calculates the share of positive and negative responses.
    
    Parameters:
    df (pd.DataFrame): The original survey DataFrame.
    question_columns (list): List of columns to transform.
    neutral_values (set): Values that represent neutrality, default is {3, 5}.
    
    Returns:
    pd.DataFrame: A transformed DataFrame with support/oppose indicators and response shares.
    """
    new_df = df_wvs.copy()
    for col in question_columns:
        valid_values = new_df[new_df[col] >= 0][col]  # Exclude negative values (missing data)
        mean_value = valid_values.mean()
        new_df[col] = new_df[col].apply(lambda x: mean_value if x < 0 else x)
    
    share_data = {}
    
    for col in question_columns:
        # Create support and oppose indicator variables
        new_df[f"{col}_support"] = new_df[col].apply(lambda x: 1 if x not in neutral_values and x > min(neutral_values) else 0)
        new_df[f"{col}_oppose"] = new_df[col].apply(lambda x: 1 if x not in neutral_values and x < min(neutral_values) else 0)
        
        # Compute shares of positive and negative responses
        total_responses = len(new_df[col].dropna())
        positive_share = new_df[f"{col}_support"].sum() / total_responses if total_responses > 0 else 0
        negative_share = new_df[f"{col}_oppose"].sum() / total_responses if total_responses > 0 else 0
        
        # Store shares
        share_data[f"{col}_positive_share"] = positive_share
        share_data[f"{col}_negative_share"] = negative_share
    
    # Convert share data to DataFrame
    share_df = pd.DataFrame([share_data])
    
    # Drop original categorical columns
    new_df.drop(columns=question_columns, inplace=True)
    
    return new_df, share_df

# Example Usage:
# Assuming survey data is loaded into a DataFrame 'df' with question columns listed in 'question_columns'



In [10]:
## Define the columns to be recoded
question_columns = ["A124_02", "A124_06", "A124_07", "A124_08", "A124_09", "C002", "E036", "E037", "E039", "F114", "F115", "F116", "F117", "F118", "F119", "F120", "F121", "F122", "F123"]
    
recode_survey_responses(df_wvs, question_columns)

KeyError: 'F114'